## Setup Example

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd.variable import Variable
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import imageio
...


## Dataset
To simplify, the PyTorch MNIST wrapper, which downloads and loads the MNIST dataset. See the [documentation](https://github.com/pytorch/vision/blob/master/torchvision/datasets/mnist.py) for more information about the interface. The default parameters will take 5,000 of the training examples and place them into a validation dataset. The data will be saved into a folder called `MNIST_data`. 

In [0]:
Data loader code

## Random Noise
Generate uniform noise from -1 to 1 with shape `[batch_size, dim]`. Implement `sample_noise` Hint: use `torch.rand`. Make sure noise is the correct shape and type:

In [0]:

def sample_noise(): 
 

# Discriminator
Our first step is to build a discriminator. Fill in the architecture: A three hidden-layer discriminative neural network.
 * Fully connected layer 
 * LeakyReLU 
 * Fully connected layer 
 * LeakyReLU 
 * Fully connected layer 

In [0]:
class DiscriminatorNet(torch.nn.Module):

    def __init__(self):
        super(DiscriminatorNet, self).__init__()    

    def forward(self, x):

        return x


# Generator
Similar like above:
 * Fully connected layer
 * `ReLU`
 * Fully connected layer
 * `ReLU`
 * Fully connected layer
 * `TanH` (to clip the image to be in the range of [-1,1])
 

In [0]:
class GeneratorNet(torch.nn.Module):

    def __init__(self):
        super(GeneratorNet, self).__init__()

    def forward(self, x):

# Optimization
Make a function that returns an `optim.Adam` optimizer

In [0]:
# Optimizers

# Loss function
loss = nn.BCELoss()


# Training a GAN!



In [0]:
def real_data_target(size):
    
    return data

def fake_data_target(size):

    return data

In [0]:
def train_discriminator(optimizer, real_data, fake_data):
    # Reset gradients
   
    # Train on Real Data
    # Calculate error and backpropagate

    # Train on Fake Data
    # Calculate error and backpropagate 
    # Update weights with gradients
    
    # Return error
    return 
def train_generator(optimizer, fake_data):
    # Reset gradients
    # Sample noise and generate fake data
    # Calculate error and backpropagate
    # Update weights with gradients
    # Return error
    return

In [0]:
logger = Logger(model_name='VGAN', data_name='MNIST')

for epoch in range(num_epochs):
    for n_batch, (real_batch,_) in enumerate(data_loader):

        # Train Discriminator
        # Generate fake data
        # Train D


        # Train Generator
        # Generate fake data
        # Train G
        # Log error

        # Model Checkpoints
        logger.save_models(generator, discriminator, epoch)